In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [2]:
def read_file(filename):
    mapping=[]
    directions=None
    get_dir=False
    with open(filename,'r') as f:
        for line in f.readlines():
            if not get_dir:
                line = line.strip('\n')
                if len(line) > 0:
                    mapping.append(list(line))
                else: 
                    get_dir=True
                    continue
            else:
                directions=line.strip()
                
    return mapping, directions

In [3]:
def get_init_pos(mp):
    max_row=len(mp)
    i=0
    while mp[-1][i] != '.':
        i+=1
    return (max_row-1, i)
    

In [8]:
def read_directions(direction):
    rotate=[]
    turns =[]
    for i,c in enumerate(direction):
        if not c.isdigit():
            rotate.append(i)
            turns.append(-1 if c == 'L' else 1)
        
    steps=[]
    for s,e  in zip([-1]+rotate, rotate+[len(direction)]):
        steps.append(int(direction[s+1:e]))
    return steps,turns

In [83]:
def move_position(steps, bearing, mapping, pos):
    rc, cc = bearing
    ir,ic = pos
    wrap=0
    new_row=None
    new_col=None
    
    max_row=len(mapping)
    max_col=len(mapping[ir])
    
    prev_pos=pos
    for i in range(1,steps+1):
        
        new_row=(ir+rc*(i+wrap))%max_row
        new_col=(ic+cc*(i+wrap))%max_col
        
        #if new_col >= max_col or mapping[new_row][new_col] not in ['.','#']:
        
        while (new_col >= max_col 
              or new_col >= len(mapping[new_row])
              or mapping[new_row][new_col] not in ['.','#']):
            wrap+=1
            new_row=(ir+rc*(i+wrap))%max_row
            new_col=(ic+cc*(i+wrap))%max_col
        
        if mapping[new_row][new_col] == '#':
            return prev_pos
        
        if mapping[new_row][new_col] == '.':
            prev_pos=[new_row,new_col]
        
                
        
    pos = [new_row,new_col]
    return pos

In [84]:
from collections import deque

bearing=[ (0,1),
          (-1,0),
          (0,-1),
          (1,0),
        ]

def part_one(filename):
    mp, d = read_file(filename)
    mp.reverse()
    
    steps, turns = read_directions(d)
    pos = get_init_pos(mp)
    b=0
   
    p = 0
    t = 0
    while p < len(turns):
        pos = move_position(steps[p], bearing[b], mp, pos)
        b = (b + turns[t])%4
        p+=1
        t+=1
    
    pos = move_position(steps[-1], bearing[b], mp, pos)
    print(pos)
    return (len(mp)-pos[0])*1000 + 4*(pos[1]+1) + b
        
    

In [85]:
%%time
part_one('input/test_22.txt')

[6, 7]
CPU times: user 627 µs, sys: 1.22 ms, total: 1.85 ms
Wall time: 1.7 ms


6032

In [86]:
%%time
part_one('input/day_22.txt')

[107, 55]
CPU times: user 25.1 ms, sys: 2.61 ms, total: 27.7 ms
Wall time: 27.5 ms


93226

# Part 2

In [125]:
class CubeFace:
    def __init__(self, coord, size=50):
        self.coord = coord
        self.size=size
        self.map = None
        
        self.neighbors=[None,None,None,None]
        
    def set_map(self, layout):
        self.map = layout
        
    def __str__(self):
        return '\n'.join([ ''.join(r) for r in self.map])
    
    def set_neighbors(self,neighs):
        self.neighbors = neighs

In [126]:
def set_up_faces(layout, size):
    layout.reverse()
    
    nrows = len(layout)//size
    faces = []
    for r in range(nrows):
        nc = len(layout[r*size])//size
        for c in range(nc):
            if layout[r*size][c*size] in ['.','#']:
                face = CubeFace((r,c), size=size)
                seg = layout[r*size:r*size+size].copy()
                new_seg = [ i[c*size:c*size+size].copy() for i in seg]
                face.set_map(new_seg)
                faces.append(face)
    
    return faces

In [127]:
tmap, tdir = read_file('input/test_22.txt')

In [128]:
tfaces = set_up_faces(tmap, 4)

In [133]:
for f in tfaces:
    print(f)
    print('\n')
   

....
.#..
....
...#


..#.
....
.#..
....


....
..#.
....
...#


....
...#
....
....


..#.
....
#...
...#


....
#...
.#..
...#


